<a href="https://colab.research.google.com/github/salemsama/nn-zero-to-hero/blob/main/makemore1/makemore_part1_E01_trigram_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# E01: train a trigram language model, i.e. take two characters as an input to predict the 3rd one.
# Feel free to use either counting or a neural net. Evaluate the loss; Did it improve over a bigram model?

# result:
# samples sound like: kellia. keon. elah. arkhwapcwwafyhcaifevina. lijamerele.
# loss: 2.3537044525146484 , still doing progress in training but it takes too long

import torch

words = open("names.txt", 'r').read().splitlines()
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
words[0]

'emma'

In [7]:
# this is trigram and exercise on bigram
# nerualnetwork version
xs = []
ys = []

for w in words:
  #we add 2 dots because we need a context (learned this idea from the video after this aka makemore 2)
  chs = ['.','.'] + list(w) + ['.']
  for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
    xs.append(stoi[ch1]*27 + stoi[ch2])
    ys.append(stoi[ch3])

xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.nelement()

print(f"xs: {xs} \nys: {ys}")

#P = (N+1).float()
#P /= P.sum(1, keepdims=True)
#g = torch.Generator().manual_seed(2147483647)

xs: tensor([  0,   5, 148,  ..., 727, 701, 726]) 
ys: tensor([ 5, 13, 13,  ..., 26, 24,  0])


In [17]:
# encodeing xs
import torch.nn.functional as F
xenc = F.one_hot(xs, num_classes = 729).float() # 729 = 27 * 27 because we have 2 chars

In [9]:
xenc[0]

tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 

In [10]:
xenc.shape # n * 729 where n is number of input aka

torch.Size([228146, 729])

In [11]:
# initilize weights
gen = torch.Generator().manual_seed(2147483647)
w = torch.randn((729,27), generator=gen, requires_grad=True)

In [37]:
for k in range(100):
  # forward pass
  logits = xenc @ w
  counts = logits.exp()
  probs  = counts / counts.sum(1, keepdim=True)

  loss = -probs[torch.arange(num), ys].log().mean()
  # backward pass
  w.grad = None # zero grad
  loss.backward()

  #update
  w.data += -10 * w.grad
  print(loss.item())


2.365194320678711
2.36506986618042
2.364945888519287
2.364821672439575
2.3646981716156006
2.364574909210205
2.3644514083862305
2.364328145980835
2.3642053604125977
2.3640825748443604
2.363959312438965
2.3638370037078857
2.3637146949768066
2.3635926246643066
2.363471031188965
2.363348960876465
2.363227605819702
2.3631060123443604
2.3629846572875977
2.362863540649414
2.3627426624298096
2.362622022628784
2.362501382827759
2.3623812198638916
2.3622608184814453
2.362140655517578
2.362020969390869
2.361901044845581
2.361781597137451
2.3616621494293213
2.3615431785583496
2.3614237308502197
2.361304998397827
2.3611862659454346
2.361067771911621
2.3609492778778076
2.3608312606811523
2.360713005065918
2.360595226287842
2.3604774475097656
2.3603599071502686
2.3602423667907715
2.3601253032684326
2.3600082397460938
2.359891414642334
2.359774589538574
2.3596582412719727
2.359541654586792
2.3594255447387695
2.359309673309326
2.3591933250427246
2.3590779304504395
2.358962059020996
2.35884690284729
2.3

In [14]:
loss.item()

3.785801887512207

In [38]:
#sampling
for i in range(5):
  out = []
  ix1 = 0
  ix2 = 0
  while True:
    xenc = [ix1*27 + ix2]
    xenc = F.one_hot(torch.tensor(xenc), num_classes=729).float()
    logits = xenc @ w
    counts = logits.exp()
    p = counts / counts.sum(1, keepdims=True)

    ix1 = ix2
    ix2 = torch.multinomial(p, num_samples=1, replacement=True, generator=gen).item()
    out.append(itos[ix2])
    if ix2 == 0:
      break
  print(''.join(out))


kellia.
keon.
elah.
arkhwapcwwafyhcaifevina.
lijamerele.
